In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import hockey_scraper
import pickle
import time
import random
pd.set_option('display.max_columns', None)

### Define Helper Functions

The below functions are created to help dynamically scrape Natural Stat Trick, the NHL API, create features, and merge the dataframes

In [2]:
#scrape NST
def get_and_format_nst_team_stats(season, sit, rate):
    #dict to convert team names from NST to team abbreviations from NHL API
    nst_to_sched = {'Anaheim Ducks': 'ANA',
                     'Arizona Coyotes': 'ARI',
                     'Boston Bruins': 'BOS',
                     'Buffalo Sabres': 'BUF',
                     'Calgary Flames': 'CGY',
                     'Carolina Hurricanes': 'CAR',
                     'Chicago Blackhawks': 'CHI',
                     'Colorado Avalanche': 'COL',
                     'Columbus Blue Jackets': 'CBJ',
                     'Dallas Stars': 'DAL',
                     'Detroit Red Wings': 'DET',
                     'Edmonton Oilers': 'EDM',
                     'Florida Panthers': 'FLA',
                     'Los Angeles Kings': 'L.A',
                     'Minnesota Wild': 'MIN',
                     'Montreal Canadiens': 'MTL',
                     'Nashville Predators': 'NSH',
                     'New Jersey Devils': 'N.J',
                     'New York Islanders': 'NYI',
                     'New York Rangers': 'NYR',
                     'Ottawa Senators': 'OTT',
                     'Philadelphia Flyers': 'PHI',
                     'Pittsburgh Penguins': 'PIT',
                     'San Jose Sharks': 'S.J',
                     'St Louis Blues': 'STL',
                     'Tampa Bay Lightning': 'T.B',
                     'Toronto Maple Leafs': 'TOR',
                     'Vancouver Canucks': 'VAN',
                     'Vegas Golden Knights': 'VGK',
                     'Washington Capitals': 'WSH',
                     'Winnipeg Jets': 'WPG'}
    #dyanmic URL
    url = 'https://www.naturalstattrick.com/games.php?fromseason={}&thruseason={}&stype=2&sit={}&loc=B&team=All&rate={}'.format(
        season,
        season,
        sit,
        rate)
    #scrape html table from webpage
    df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
    #reset index
    df.reset_index(inplace = True)
    #format date
    df['Date'] = df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    #add team game number
    df['Game_Number'] = df.groupby('Team').cumcount() + 1
    #replcate Team name with team abbreviation
    df = df.replace({'Team': nst_to_sched})
    #add team key to merge with game results df
    df['Team_Key'] = df['Team'].astype(str)+'_'+df['Date'].astype(str)
    return df

In [3]:
#merge 5v5, PP, and PK team game logs from NST
def merge_team_stats(primary_df, pp_df, pk_df):
    primary_df = primary_df.merge(pk_df[['Team_Key', 'TOI', 'xGA', 'GA']], on = 'Team_Key', how = 'left', suffixes = ('','_pk') )
    primary_df = primary_df.merge(pp_df[['Team_Key', 'TOI', 'xGF', 'GF']], on = 'Team_Key', how = 'left', suffixes = ('','_pp') )
    return primary_df

Feature Engineering  
See Exploratory Data Analysis Notebook for more commentary on the Features

In [4]:
#calculate team features. Number of rolling games used is dynamic to test if different numbers of game work better.
def calculate_team_features(df, rolling_games = 20):

    df[f'sum_rolling{rolling_games}_TOI_5v5'] = df.groupby('Team')['TOI'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    df[f'sum_rolling{rolling_games}_FF_5v5'] = df.groupby('Team')['FF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_FA_5v5'] = df.groupby('Team')['FA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_GF_5v5'] = df.groupby('Team')['GF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_GA_5v5'] = df.groupby('Team')['GA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_xGF_5v5'] = df.groupby('Team')['xGF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_xGA_5v5'] = df.groupby('Team')['xGA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_SF_5v5'] = df.groupby('Team')['SF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'last_{rolling_games}_FF%_5v5'] = df[f'sum_rolling{rolling_games}_FF_5v5']*100/ (df[f'sum_rolling{rolling_games}_FF_5v5']+df[f'sum_rolling{rolling_games}_FA_5v5'])
    df[f'last_{rolling_games}_GF%_5v5'] = df[f'sum_rolling{rolling_games}_GF_5v5']*100/ (df[f'sum_rolling{rolling_games}_GF_5v5']+df[f'sum_rolling{rolling_games}_GA_5v5'])
    df[f'last_{rolling_games}_xGF%_5v5'] = df[f'sum_rolling{rolling_games}_xGF_5v5']*100/ (df[f'sum_rolling{rolling_games}_xGF_5v5']+df[f'sum_rolling{rolling_games}_xGA_5v5'])
    df[f'last_{rolling_games}_SH%'] = df[f'sum_rolling{rolling_games}_GF_5v5']*100 / df[f'sum_rolling{rolling_games}_SF_5v5']
    
    
    #fix NaNs in pp and pk features. If team wasnt on PP or PK in a game that game is missing from dataframe.
    df['TOI_pp'] = np.where(df['TOI_pp'].isna(), 0, df['TOI_pp'])
    df['TOI_pk'] = np.where(df['TOI_pk'].isna(), 0, df['TOI_pk'])
    df['xGF_pp'] = np.where(df['xGF_pp'].isna(), 0, df['xGF_pp'])
    df['GF_pp']  = np.where(df['GF_pp'].isna(), 0, df['GF_pp'])
    df['xGA_pk'] = np.where(df['xGA_pk'].isna(), 0, df['xGA_pk'])
    df['GA_pk']  = np.where(df['GA_pk'].isna(), 0, df['GA_pk'])
    
    #pp features
    df[f'sum_rolling{rolling_games}_TOI_pp'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_xGF_pp'] = df.groupby('Team')['xGF_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_GF_pp'] = df.groupby('Team')['GF_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'last{rolling_games}_pp_TOI_per_game'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).mean().shift())
    df[f'last{rolling_games}_xGF_per_min_pp'] = df[f'sum_rolling{rolling_games}_xGF_pp'] / df[f'sum_rolling{rolling_games}_TOI_pp'] 
    df[f'last{rolling_games}_GF_per_min_pp'] = df[f'sum_rolling{rolling_games}_GF_pp'] / df[f'sum_rolling{rolling_games}_TOI_pp'] 
    
    
    #pk features
    df[f'sum_rolling{rolling_games}_TOI_pk'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_xGA_pk'] = df.groupby('Team')['xGA_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'sum_rolling{rolling_games}_GA_pk'] = df.groupby('Team')['GA_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df[f'last{rolling_games}_pk_TOI_per_game'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).mean().shift())
    df[f'last{rolling_games}_xGA_per_min_pk'] = df[f'sum_rolling{rolling_games}_xGA_pk'] / df[f'sum_rolling{rolling_games}_TOI_pk'] 
    df[f'last{rolling_games}_GA_per_min_pk'] = df[f'sum_rolling{rolling_games}_GA_pk'] / df[f'sum_rolling{rolling_games}_TOI_pk'] 
    
    #to get back to back category
    df['Last_Game_Date'] = df.groupby('Team')['Date'].shift()
    df['Days_Since_Last_Game'] = df['Date'] - df['Last_Game_Date']
    df['B2B'] = np.where(df['Days_Since_Last_Game'] == '1 days', 1, 0)
    
    

    
    return df

In [5]:
#scrape data from NHL API via hockey_scraper to get official results. 
def get_game_results(season_start, season_end):
    sched_df = hockey_scraper.scrape_schedule(season_start, season_end)
    sched_df['Home_Team_Won'] = np.where(sched_df['home_score'] > sched_df['away_score'], 1, 0)
    #create keys for home and away team in order to import their features
    sched_df['Home_Team_Key'] = sched_df['home_team'].astype(str)+'_'+sched_df['date'].astype(str)
    sched_df['Away_Team_Key'] = sched_df['away_team'].astype(str)+'_'+sched_df['date'].astype(str)
    return sched_df

In [6]:
#merge team features, goalies feature, and Elo feature to the schedule DF. This DF will be used for modeling.
def merge_starters_and_features(game_results_df, goalies_df, features_df, elo, feature_columns, goalie_feature_columns):
    goalies_df = goalies_df[goalies_df['TOI'] >=28.5]
    df = game_results_df.merge(goalies_df[goalie_feature_columns].add_prefix('home_'), left_on = 'Home_Team_Key', right_on = 'home_Team_Key', how = 'left').rename(columns ={'home_Name':'home_goalie'}).drop(columns = 'home_Team_Key')
    df = df.merge(goalies_df[goalie_feature_columns].add_prefix('away_'), left_on = 'Away_Team_Key', right_on = 'away_Team_Key', how = 'left').rename(columns ={'away_Name':'away_goalie'}).drop(columns = 'away_Team_Key')
    df = df.merge(features_df[feature_columns].add_prefix('home_'), left_on = 'Home_Team_Key', right_on = 'home_Team_Key', how = 'left')
    df = df.merge(features_df[feature_columns].add_prefix('away_'), left_on = 'Away_Team_Key', right_on = 'away_Team_Key', how = 'left')
    df = df.merge(elo[['elo_Team_Key', 'Rating.A.Pre']].add_prefix('home_'), left_on='Home_Team_Key', right_on='home_elo_Team_Key', how = 'left').drop(columns = 'home_elo_Team_Key')
    df = df.merge(elo[['elo_Team_Key', 'Rating.A.Pre']].add_prefix('away_'), left_on='Away_Team_Key', right_on='away_elo_Team_Key', how= 'left').drop(columns = 'away_elo_Team_Key')
    
    #categorize B2B
    conditions = [((df['home_B2B'] == 0) & (df['away_B2B'] == 0)),
                  ((df['home_B2B'] == 1) & (df['away_B2B'] == 0)),
                  ((df['home_B2B'] == 0) & (df['away_B2B'] == 1)),
                  ((df['home_B2B'] == 1) & (df['away_B2B'] == 1))
                 ]
    
    choices = ['Neither',
               'Home_only',
               'Away_only',
               'Both']

    df['B2B_Status'] = np.select(conditions, choices)
    
    
    #season
    conditions = [((df['date'] >= '2017-10-04') & (df['date'] <= '2018-04-08')),
                  ((df['date'] >= '2018-10-03') & (df['date'] <= '2019-04-06')),
                  ((df['date'] >= '2019-10-02') & (df['date'] <= '2020-03-12')),
                  ((df['date'] >= '2021-01-13') & (df['date'] <= '2021-06-29'))
                 ]
    
    choices = ['2017-2018',
               '2018-2019',
               '2019-2020',
               '2020-2021']

    df['Season'] = np.select(conditions, choices)

    return df

In [7]:
#goalie feature columns
goalie_feature_columns = ['Team_Key', 'Name', 'Goalie_FenwickSV%', 'Goalie_GSAx/60', 'Goalie_HDCSV%']

### Get ELO Data

Credit to [Neil Paine](https://github.com/NeilPaine538/NHL-Player-And-Team-Ratings) for calculating and providing the data.

In [8]:
elo = pd.read_csv('https://raw.githubusercontent.com/NeilPaine538/NHL-Player-And-Team-Ratings/master/nhl_elo_historical.csv')

In [9]:
elo = elo[elo['Date'] > '2013']

In [10]:
elo.head()

,Game.ID,Date,Year,Team.A,Franch.A,Rating.A.Pre,Rating.A.Post,Goals.A,Team.B,Franch.B,Rating.B.Pre,Rating.B.Post,Goals.B,OT,Playoff,Neutral,Home,Win,Margin,Elo_diff,HomeIce,Forecast,Delta,Multiplier,Shift
104708,201300119BOS,2013-01-19,2013,Boston Bruins,BOS,1543.19,1546.31,3.0,New York Rangers,NYR,1529.50,1526.38,1.0,NaN,0,0,1,1.0,2,63.69,1,59.1%,0.409,1.2682,3.1149
104709,201300119BOS,2013-01-19,2013,New York Rangers,NYR,1529.50,1526.38,1.0,Boston Bruins,BOS,1543.19,1546.31,3.0,NaN,0,0,0,0.0,-2,-63.69,-1,40.9%,-0.409,1.2682,-3.1149
104710,201300119DAL,2013-01-19,2013,Dallas Stars,DAL,1497.23,1499.51,4.0,Phoenix Coyotes,ARI,1528.15,1525.87,3.0,NaN,0,0,1,1.0,1,19.08,1,52.7%,0.473,0.8048,2.2820
104711,201300119DAL,2013-01-19,2013,Phoenix Coyotes,ARI,1528.15,1525.87,3.0,Dallas Stars,DAL,1497.23,1499.51,4.0,NaN,0,0,0,0.0,-1,-19.08,-1,47.3%,-0.473,0.8048,-2.2820
104712,201300119FLA,2013-01-19,2013,Florida Panthers,FLA,1483.35,1487.87,5.0,Carolina Hurricanes,CAR,1487.70,1483.19,1.0,NaN,0,0,1,1.0,4,45.65,1,56.5%,0.435,1.7317,4.5163


In [11]:
elo_conversion = {'VEG' : 'VGK', 'NJD': 'N.J',  'SJS': 'S.J', 'TBL' : 'T.B',  'LAK' : 'L.A'}

In [12]:
elo = elo.replace({'Franch.A': elo_conversion})

In [13]:
elo.head()

,Game.ID,Date,Year,Team.A,Franch.A,Rating.A.Pre,Rating.A.Post,Goals.A,Team.B,Franch.B,Rating.B.Pre,Rating.B.Post,Goals.B,OT,Playoff,Neutral,Home,Win,Margin,Elo_diff,HomeIce,Forecast,Delta,Multiplier,Shift
104708,201300119BOS,2013-01-19,2013,Boston Bruins,BOS,1543.19,1546.31,3.0,New York Rangers,NYR,1529.50,1526.38,1.0,NaN,0,0,1,1.0,2,63.69,1,59.1%,0.409,1.2682,3.1149
104709,201300119BOS,2013-01-19,2013,New York Rangers,NYR,1529.50,1526.38,1.0,Boston Bruins,BOS,1543.19,1546.31,3.0,NaN,0,0,0,0.0,-2,-63.69,-1,40.9%,-0.409,1.2682,-3.1149
104710,201300119DAL,2013-01-19,2013,Dallas Stars,DAL,1497.23,1499.51,4.0,Phoenix Coyotes,ARI,1528.15,1525.87,3.0,NaN,0,0,1,1.0,1,19.08,1,52.7%,0.473,0.8048,2.2820
104711,201300119DAL,2013-01-19,2013,Phoenix Coyotes,ARI,1528.15,1525.87,3.0,Dallas Stars,DAL,1497.23,1499.51,4.0,NaN,0,0,0,0.0,-1,-19.08,-1,47.3%,-0.473,0.8048,-2.2820
104712,201300119FLA,2013-01-19,2013,Florida Panthers,FLA,1483.35,1487.87,5.0,Carolina Hurricanes,CAR,1487.70,1483.19,1.0,NaN,0,0,1,1.0,4,45.65,1,56.5%,0.435,1.7317,4.5163


In [14]:
elo['Franch.A'].value_counts()

PIT    773
T.B    771
BOS    766
WSH    755
STL    755
CHI    751
S.J    744
NYR    742
NSH    736
NYI    731
ANA    731
MTL    729
DAL    724
MIN    720
L.A    719
COL    716
WPG    706
TOR    705
OTT    703
CBJ    703
PHI    700
CAR    699
DET    698
CGY    696
VAN    692
EDM    688
FLA    681
ARI    675
N.J    670
BUF    665
VGK    352
Name: Franch.A, dtype: int64

In [15]:
elo['elo_Team_Key'] = elo['Franch.A'].astype(str)+'_'+elo['Date'].astype(str)

### Get Goalie Data

In [22]:
#import dictionary with goalie names and IDs from NHL API
infile = open("data/goalie_ids.pickle",'rb')
goalie_ids = pickle.load(infile)
infile.close()

In [23]:
## scrape season long stats to get name of all goalies who played in time frame
goalie_list = pd.read_html('https://www.naturalstattrick.com/playerteams.php?fromseason=20182019&thruseason=20202021&stype=2&sit=5v5&score=all&stdoi=g&rate=n&team=ALL&pos=S&loc=B&toi=0&gpfilt=none&fd=&td=&tgp=410&lines=single&draftteam=ALL')[0]

In [24]:
#find which goalies are missing from goalie_ids dictionary
missing_goalies2 = [g for g in list(goalie_list['Player']) if g not in goalie_ids.keys() ]

In [25]:
## Cal Petersen already in Data Dictionary as Calvin Petersen 
missing_goalies2.remove('Cal Petersen')

In [26]:
missing_goalies2

[]

In [27]:
def goalie_features(df, rolling_games = 40):
    rolling_games = rolling_games
    min_games = 10
    df['Date'] = df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    df['Team_Key'] = df['Team'].astype(str)+'_'+df['Date'].astype(str)
    
    df['Rolling_TOI'] = df.groupby('ID')['TOI'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_FA'] = df.groupby('ID')['FA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_SA'] = df.groupby('ID')['SA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_GA'] = df.groupby('ID')['GA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_xGA'] = df.groupby('ID')['xGA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_HDCA'] = df.groupby('ID')['HDCA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    df['Rolling_HDGA'] = df.groupby('ID')['HDGA'].transform(lambda x: x.rolling(rolling_games, min_games).sum().shift())
    
    df['Goalie_FenwickSV%'] =  (df['Rolling_FA'] - df['Rolling_GA']) /  df['Rolling_FA']
    df['Goalie_GSAx'] = df['Rolling_xGA'] - df['Rolling_GA']
    df['Goalie_GSAx/60'] =  df['Goalie_GSAx']*60 /  df['Rolling_TOI']
    df['Goalie_HDCSV%'] = (df['Rolling_HDCA'] - df['Rolling_HDGA'] ) / df['Rolling_HDCA'] 
    return df

In [28]:
# only scrape at most 2 seasons at a time
def get_goalie_data(goalie_ids, start_year, end_year):
    counter = 0
    for name, gid in goalie_ids.items():

        sequence = [x/10 for x in range(60, 120)]
        time.sleep(random.choice(sequence))
        url = 'https://www.naturalstattrick.com/playerreport.php?fromseason={}&thruseason={}&playerid={}&sit=all&stype=2&stdoi=oi&rate=n&v=g'.format(start_year, end_year, gid)
        #due to number of http requests, NST may ban your IP before the loop finishes. I needed to use a VPN to get around this. If IP gets banned, this function will still return the current DF and you can call the function again and pass in an updated goalie dictionary to get the rest
        try:
            individual_df = pd.read_html(url)[0]
            individual_df['Name'] = name
            individual_df['ID'] = gid
        except:
            print(f'Ended before {name}')
            return all_goalies4 

        if counter == 0:
            all_goalies4 = individual_df
            print(name)
            print(counter)
        elif counter != 0:
            all_goalies4 = pd.concat([all_goalies4, individual_df])
            print(name)
            print(counter)


        counter +=1
    
    return all_goalies4

In [29]:
goalies_161718 = get_goalie_data(goalie_ids, 20162017, 20172018)

Scott Wedgewood
0
Aaron Dell
1
Mackenzie Blackwood
2
Cory Schneider
3
Semyon Varlamov
4
Ilya Sorokin
5
Keith Kinkaid
6
Igor Shesterkin
7
Alexandar Georgiev
8
Brian Elliott
9
Alex Lyon
10
Carter Hart
11
Emil Larmi
12
Tristan Jarry
13
Casey DeSmith
14
Jaroslav Halak
15
Tuukka Rask
16
Michael Houser
17
Ukko-Pekka Luukkonen
18
Dustin Tokarski
19
Carter Hutton
20
Linus Ullmark
21
Charlie Lindgren
22
Carey Price
23
Jake Allen
24
Cayden Primeau
25
Joey Daccord
26
Anton Forsberg
27
Matt Murray
28
Marcus Hogberg
29
Filip Gustavsson
30
Frederik Andersen
31
Jack Campbell
32
David Rittich
33
James Reimer
34
Petr Mrazek
35
Alex Nedeljkovic
36
Philippe Desrosiers
37
Sam Montembeault
38
Sergei Bobrovsky
39
Chris Driedger
40
Spencer Knight
41
Christopher Gibson
42
Curtis McElhinney
43
Andrei Vasilevskiy
44
Craig Anderson
45
Vitek Vanecek
46
Ilya Samsonov
47
Malcolm Subban
48
Collin Delia
49
Kevin Lankinen
50
Thomas Greiss
51
Jonathan Bernier
52
Kasimir Kaskisuo
53
Pekka Rinne
54
Juuse Saros
55
Jordan 

In [30]:
goalies_161718.to_csv('data/goalie_logs_1617_1718')

In [31]:
goalies_181920 = get_goalie_data(goalie_ids, 20182019, 20192020)

Scott Wedgewood
0
Aaron Dell
1
Mackenzie Blackwood
2
Cory Schneider
3
Semyon Varlamov
4
Ilya Sorokin
5
Keith Kinkaid
6
Igor Shesterkin
7
Alexandar Georgiev
8
Brian Elliott
9
Alex Lyon
10
Carter Hart
11
Emil Larmi
12
Tristan Jarry
13
Casey DeSmith
14
Jaroslav Halak
15
Tuukka Rask
16
Michael Houser
17
Ukko-Pekka Luukkonen
18
Dustin Tokarski
19
Carter Hutton
20
Linus Ullmark
21
Charlie Lindgren
22
Carey Price
23
Jake Allen
24
Cayden Primeau
25
Joey Daccord
26
Anton Forsberg
27
Matt Murray
28
Marcus Hogberg
29
Filip Gustavsson
30
Frederik Andersen
31
Jack Campbell
32
David Rittich
33
James Reimer
34
Petr Mrazek
35
Alex Nedeljkovic
36
Philippe Desrosiers
37
Sam Montembeault
38
Sergei Bobrovsky
39
Chris Driedger
40
Spencer Knight
41
Christopher Gibson
42
Curtis McElhinney
43
Andrei Vasilevskiy
44
Craig Anderson
45
Vitek Vanecek
46
Ilya Samsonov
47
Malcolm Subban
48
Collin Delia
49
Kevin Lankinen
50
Thomas Greiss
51
Jonathan Bernier
52
Kasimir Kaskisuo
53
Pekka Rinne
54
Juuse Saros
55
Jordan 

In [ ]:
goalies_181920.to_csv('data/goalie_logs_1819_1920')

In [33]:
goalies_2021 = get_goalie_data(goalie_ids, 20202021, 20202021)

Scott Wedgewood
0
Aaron Dell
1
Mackenzie Blackwood
2
Cory Schneider
3
Semyon Varlamov
4
Ilya Sorokin
5
Keith Kinkaid
6
Igor Shesterkin
7
Alexandar Georgiev
8
Brian Elliott
9
Alex Lyon
10
Carter Hart
11
Emil Larmi
12
Tristan Jarry
13
Casey DeSmith
14
Jaroslav Halak
15
Tuukka Rask
16
Michael Houser
17
Ukko-Pekka Luukkonen
18
Dustin Tokarski
19
Carter Hutton
20
Linus Ullmark
21
Charlie Lindgren
22
Carey Price
23
Jake Allen
24
Cayden Primeau
25
Joey Daccord
26
Anton Forsberg
27
Matt Murray
28
Marcus Hogberg
29
Filip Gustavsson
30
Frederik Andersen
31
Jack Campbell
32
David Rittich
33
James Reimer
34
Petr Mrazek
35
Alex Nedeljkovic
36
Philippe Desrosiers
37
Sam Montembeault
38
Sergei Bobrovsky
39
Chris Driedger
40
Spencer Knight
41
Christopher Gibson
42
Curtis McElhinney
43
Andrei Vasilevskiy
44
Craig Anderson
45
Vitek Vanecek
46
Ilya Samsonov
47
Malcolm Subban
48
Collin Delia
49
Kevin Lankinen
50
Thomas Greiss
51
Jonathan Bernier
52
Kasimir Kaskisuo
53
Pekka Rinne
54
Juuse Saros
55
Jordan 

In [34]:
goalies_2021.to_csv('data/goalie_logs_2021')

In [35]:
goalies_all_C = pd.concat([goalies_161718, goalies_181920, goalies_2021])

In [36]:
goalie_features_dfC = goalie_features(goalies_all_C)

In [37]:
goalie_features_dfC

,Game,Team,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCGF,SCGA,SCGF%,HDCF,HDCA,HDCF%,HDGF,HDGA,HDGF%,MDCF,MDCA,MDCF%,MDGF,MDGA,MDGF%,LDCF,LDCA,LDCF%,LDGF,LDGA,LDGF%,On-Ice SH%,On-Ice SV%,PDO,Off. Zone Starts,Neu. Zone Starts,Def. Zone Starts,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %,Name,ID,Date,Team_Key,Rolling_TOI,Rolling_FA,Rolling_SA,Rolling_GA,Rolling_xGA,Rolling_HDCA,Rolling_HDGA,Goalie_FenwickSV%,Goalie_GSAx,Goalie_GSAx/60,Goalie_HDCSV%
0,2017-10-30 ARI at PHI,ARI,64.666667,58,60,49.15,42,48,46.67,33,31,51.56,4,3,57.14,2.98,2.27,56.78,32,21,60.38,3,3,50,13,8,61.9,1,2,33.33,19,13,59.38,2,1,66.67,18,35,33.96,1,0,100.00,12.12,90.32,1.024,1,4,0,0,100.00,24,23,25,48.98,Scott Wedgewood,8475809,2017-10-30,ARI_2017-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-10-31 ARI at DET,ARI,58.333333,54,63,46.15,41,53,43.62,32,39,45.07,3,4,42.86,1.73,2.50,41,28,32,46.67,1,2,33.33,11,15,42.31,1,1,50,17,17,50,0,1,0.00,24,29,45.28,2,2,50.00,9.38,89.74,0.991,1,4,0,0,100.00,18,22,24,42.86,Scott Wedgewood,8475809,2017-10-31,ARI_2017-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-11-06 ARI at WSH,ARI,63.883333,49,75,39.52,38,60,38.78,26,40,39.39,2,3,40,1.81,2.84,38.95,20,31,39.22,2,1,66.67,8,10,44.44,2,1,66.67,12,21,36.36,0,0,-,26,37,41.27,0,2,0.00,7.69,92.5,1.002,1,4,0,0,100.00,14,18,26,35,Scott Wedgewood,8475809,2017-11-06,ARI_2017-11-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-11-14 ARI at WPG,ARI,34.866667,32,26,55.17,20,21,48.78,18,17,51.43,1,1,50,1.15,1.41,45,17,15,53.13,0,1,0,5,7,41.67,0,1,0,12,8,60,0,0,-,14,9,60.87,1,0,100.00,5.56,94.12,0.997,0,2,0,0,-,10,14,7,58.82,Scott Wedgewood,8475809,2017-11-14,ARI_2017-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-11-22 S.J at ARI,ARI,44.333333,32,32,50,23,24,48.94,19,16,54.29,1,2,33.33,1.22,1.41,46.35,12,16,42.86,1,2,33.33,8,7,53.33,1,2,33.33,4,9,30.77,0,0,-,15,15,50,0,0,-,5.26,87.5,0.928,0,3,1,0,0.00,9,13,22,29.03,Scott Wedgewood,8475809,2017-11-22,ARI_2017-11-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2021-04-09 ARI at VGK,ARI,18.416667,18,10,64.29,12,5,70.59,7,4,63.64,3,1,75,0.63,0.44,58.94,10,8,55.56,3,1,75,3,3,50,1,1,50,7,5,58.33,2,0,100,7,0,100,0,0,-,42.86,75,1.179,0,2,0,0,-,5,10,2,71.43,Ivan Prosvetov,8481031,2021-04-09,ARI_2021-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-12 ARI at COL,ARI,57.966667,58,51,53.21,47,32,59.49,33,19,63.46,2,3,40,2.87,1.80,61.36,19,29,39.58,2,3,40,9,7,56.25,1,1,50,10,22,31.25,1,2,33.33,36,21,63.16,0,0,-,6.06,84.21,0.903,0,4,0,0,-,18,18,22,45,Ivan Prosvetov,8481031,2021-04-12,ARI_2021-04-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021-02-11 S.J at L.A,S.J,9.183333,10,6,62.5,10,6,62.5,7,5,58.33,0,0,-,0.59,0.15,80.17,3,1,75,0,0,-,3,0,100,0,0,-,0,1,0,0,0,-,7,2,77.78,0,0,-,0,100,1,0,1,0,0,-,3,2,0,100,Alexei Melnichuk,8482246,2021-02-11,S.J_2021-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-08 ARI at S.J,S.J,61.400000,62,45,57.94,55,39,58.51,46,32,58.97,3,5,37.50,3.64,3.07,54.26,38,28,57.58,3,4,42.86,16,12,57.14,2,1,66.67,22,16,57.89,1,3,25.00,22,16,57.89,0,1,0.00,6.52,84.38,0.909,0,5,0,0,-,23,22,18,56.1,Alexei Melnichuk,8482246,2021-05-08,S.J_2021-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
pickle_out = open("goalie_game_log_df.pickle","wb")
pickle.dump(goalie_features_dfC, pickle_out)
pickle_out.close()


In [33]:
goalie_features_dfC.to_csv('data/goalie_game_logs_C.csv')

### Determine Goalie Stats to Impute for Goalies Having Player Less Than 10 Games

In [53]:
goalie_features_dfC.isna().sum()

Game                    0
Team                    0
TOI                     0
CF                      0
CA                      0
                     ... 
Rolling_HDGA         1233
Goalie_FenwickSV%    1233
Goalie_GSAx          1233
Goalie_GSAx/60       1233
Goalie_HDCSV%        1233
Length: 69, dtype: int64

In [54]:
#set df for goalies who hadnt played 10 games at that point
ig_df = goalie_features_dfC[(goalie_features_dfC['Goalie_FenwickSV%'].isna()) & (goalie_features_dfC['Date'] >= '2017-10-04') & (goalie_features_dfC['Date']<'2021')]

In [55]:
ig_df[ig_df['Date'].apply(lambda x: x.year) == 2021]

,Game,Team,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCGF,SCGA,SCGF%,HDCF,HDCA,HDCF%,HDGF,HDGA,HDGF%,MDCF,MDCA,MDCF%,MDGF,MDGA,MDGF%,LDCF,LDCA,LDCF%,LDGF,LDGA,LDGF%,On-Ice SH%,On-Ice SV%,PDO,Off. Zone Starts,Neu. Zone Starts,Def. Zone Starts,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %,Name,ID,Date,Team_Key,Rolling_TOI,Rolling_FA,Rolling_SA,Rolling_GA,Rolling_xGA,Rolling_HDCA,Rolling_HDGA,Goalie_FenwickSV%,Goalie_GSAx,Goalie_GSAx/60,Goalie_HDCSV%


In [56]:
ig_TOI = ig_df['TOI'].sum()
ig_FA = ig_df['FA'].sum()
ig_GA = ig_df['GA'].sum()
ig_xGA = ig_df['xGA'].sum()
ig_HDCA = ig_df['HDCA'].sum()
ig_HDGA = ig_df['HDGA'].sum()

In [57]:
ig_FenwickSV = (ig_FA - ig_GA) /ig_FA
ig_GSAx = ig_xGA - ig_GA
ig_GSAx60 = (ig_GSAx*60) / ig_TOI
ig_HDCSV = (ig_HDCA - ig_HDGA )/ ig_HDCA

In [58]:
#experience goalie df for comparison
eg_df = goalie_features_dfC[(~goalie_features_dfC['Goalie_FenwickSV%'].isna()) & (goalie_features_dfC['Date'] >= '2017-10-04') & (goalie_features_dfC['Date']<'2021')]

In [59]:
eg_TOI = eg_df['TOI'].sum()
eg_FA = eg_df['FA'].sum()
eg_GA = eg_df['GA'].sum()
eg_xGA = eg_df['xGA'].sum()
eg_HDCA = eg_df['HDCA'].sum()
eg_HDGA = eg_df['HDGA'].sum()
eg_FenwickSV = (eg_FA - eg_GA) /eg_FA
eg_GSAx = eg_xGA - eg_GA
eg_GSAx60 = (eg_GSAx*60) / eg_TOI
eg_HDCSV = (eg_HDCA - eg_HDGA )/ eg_HDCA

In [60]:
display(ig_FenwickSV)
display(eg_FenwickSV)


0.9350856996674342

0.9350482962167963

In [61]:
display(ig_GSAx60)
display(eg_GSAx60)


-0.2556941982582863

-0.22148441971590457

In [62]:
goalie_features_dfB['Goalie_GSAx/60'].std()

NameError: name 'goalie_features_dfB' is not defined

In [ ]:
display(ig_HDCSV )
display(eg_HDCSV)


In [ ]:
ig = [ig_FenwickSV,
ig_GSAx ,
ig_GSAx60 ,
ig_HDCSV ]

### Calculating Weighted Rolling Features
This is a work in progress

In [455]:
# def calculate_weighted_team_features(df, rolling_games = 20):
#     weights = np.arange(1,rolling_games)
#     df[f'sum_rolling{rolling_games}_TOI_5v5'] = df.groupby('Team')['TOI'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
#     df[f'sum_rolling{rolling_games}_FF_5v5'] = df.groupby('Team')['FF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'sum_rolling{rolling_games}_FA_5v5'] = df.groupby('Team')['FA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'sum_rolling{rolling_games}_GF_5v5'] = df.groupby('Team')['GF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'sum_rolling{rolling_games}_GA_5v5'] = df.groupby('Team')['GA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'sum_rolling{rolling_games}_xGF_5v5'] = df.groupby('Team')['xGF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'sum_rolling{rolling_games}_xGA_5v5'] = df.groupby('Team')['xGA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'sum_rolling{rolling_games}_SF_5v5'] = df.groupby('Team')['SF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'last_{rolling_games}_FF%_5v5'] = df[f'sum_rolling{rolling_games}_FF_5v5']*100/ (df[f'sum_rolling{rolling_games}_FF_5v5']+df[f'sum_rolling{rolling_games}_FA_5v5'])
#     df[f'last_{rolling_games}_GF%_5v5'] = df[f'sum_rolling{rolling_games}_GF_5v5']*100/ (df[f'sum_rolling{rolling_games}_GF_5v5']+df['sum_rolling20_GA_5v5'])
#     df[f'last_{rolling_games}_xGF%_5v5'] = df['sum_rolling20_xGF_5v5']*100/ (df['sum_rolling20_xGF_5v5']+df[f'sum_rolling{rolling_games}_GA_5v5'])
#     df[f'last_{rolling_games}_SH%'] = df[f'sum_rolling{rolling_games}_GF_5v5']*100 / df[f'sum_rolling{rolling_games}_SF_5v5']
    
    
#     #fix NaNs in pp and pk features
#     df['TOI_pp'] = np.where(df['TOI_pp'].isna(), 0, df['TOI_pp'])
#     df['TOI_pk'] = np.where(df['TOI_pk'].isna(), 0, df['TOI_pk'])
#     df['xGF_pp'] = np.where(df['xGF_pp'].isna(), 0, df['xGF_pp'])
#     df['xGA_pk'] = np.where(df['xGA_pk'].isna(), 0, df['xGA_pk'])
    
#     #pp features
#     df[f'sum_rolling{rolling_games}_TOI_pp'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'sum_rolling{rolling_games}_xGF_pp'] = df.groupby('Team')['xGF_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'last{rolling_games}_pp_TOI_per_game'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).mean().shift())
#     df[f'last{rolling_games}_xGF_per_min_pp'] = df[f'sum_rolling{rolling_games}_xGF_pp'] / df[f'sum_rolling{rolling_games}_TOI_pp'] 
    
#     #pk features
#     df[f'sum_rolling{rolling_games}_TOI_pk'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'sum_rolling{rolling_games}_xGA_pk'] = df.groupby('Team')['xGA_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
#     df[f'last{rolling_games}_pk_TOI_per_game'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).mean().shift())
#     df[f'last{rolling_games}_xGA_per_min_pk'] = df[f'sum_rolling{rolling_games}_xGA_pk'] / df[f'sum_rolling{rolling_games}_TOI_pk'] 
    
#     #to get back to back category
#     df['Last_Game_Date'] = df.groupby('Team')['Date'].shift()
#     df['Days_Since_Last_Game'] = df['Date'] - df['Last_Game_Date']
#     df['B2B'] = np.where(df['Days_Since_Last_Game'] == '1 days', 1, 0)
    
#     return df

In [471]:
# df = primary
# w = np.arange(1, 21)
# rolling_games = 20
# df[f'sum_rolling{rolling_games}_FF_5v5'] = df.groupby('Team')['FF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).apply(lambda x: (x * w).sum()).shift())
# df[f'sum_rolling{rolling_games}_FA_5v5'] = df.groupby('Team')['FA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).apply(lambda x: (x * w).sum()).shift())
# df[f'last_{rolling_games}_FF%_5v5'] = df[f'sum_rolling{rolling_games}_FF_5v5']*100/ (df[f'sum_rolling{rolling_games}_FF_5v5']+df[f'sum_rolling{rolling_games}_FA_5v5'])


### Get Data With No Scoring and Venue Adjustments

In [518]:
primary1617 = get_and_format_nst_team_stats('20162017','5v5', 'n')
pp1617 = get_and_format_nst_team_stats('20162017','pp', 'n')
pk1617 = get_and_format_nst_team_stats('20162017','pk', 'n')

In [519]:
primary1718 = get_and_format_nst_team_stats('20172018','5v5', 'n')
pp1718 = get_and_format_nst_team_stats('20172018','pp', 'n')
pk1718 = get_and_format_nst_team_stats('20172018','pk', 'n')

In [520]:
features1617 = merge_team_stats(primary1617, pp1617, pk1617)
features1718 = merge_team_stats(primary1718, pp1718, pk1718)
features1819 = merge_team_stats(primary1819, pp1819, pk1819)
features1920 = merge_team_stats(primary1920, pp1920, pk1920)
features2021 = merge_team_stats(primary2021, pp2021, pk2021)

In [521]:
team_stats_all_seasons = pd.concat([features1617, features1718, features1819, features1920, features2021]).sort_values('Date')

In [522]:
pd.options.display.max_rows = 100
team_stats_all_seasons.isna().sum()

Game              0
Team              0
Unnamed: 2        0
TOI               0
CF                0
CA                0
CF%               0
FF                0
FA                0
FF%               0
SF                0
SA                0
SF%               0
GF                0
GA                0
GF%             206
xGF               0
xGA               0
xGF%             50
SCF               0
SCA               0
SCF%              0
HDCF              0
HDCA              0
HDCF%            50
HDSF              0
HDSA              0
HDSF%            50
HDGF              0
HDGA              0
HDGF%          1362
HDSH%           101
HDSV%           101
MDCF              0
MDCA              0
MDCF%             0
MDSF              0
MDSA              0
MDSF%             2
MDGF              0
MDGA              0
MDGF%          4044
MDSH%            44
MDSV%            44
LDCF              0
LDCA              0
LDCF%             0
LDSF              0
LDSA              0
LDSF%             0


In [523]:
for games in [3,5,10,20,30]:
    team_stats_all_seasons = calculate_team_features(team_stats_all_seasons, games)

In [524]:
team_stats_all_seasons.tail()

,Game,Team,Unnamed: 2,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,...,last_10_xGF%_5v5,last_10_SH%,sum_rolling10_TOI_pp,sum_rolling10_xGF_pp,last10_pp_TOI_per_game,last10_xGF_per_min_pp,sum_rolling10_TOI_pk,sum_rolling10_xGA_pk,last10_pk_TOI_per_game,last10_xGA_per_min_pk,sum_rolling20_TOI_5v5,sum_rolling20_FF_5v5,sum_rolling20_FA_5v5,sum_rolling20_GF_5v5,sum_rolling20_GA_5v5,sum_rolling20_xGF_5v5,sum_rolling20_xGA_5v5,sum_rolling20_SF_5v5,last_20_FF%_5v5,last_20_GF%_5v5,last_20_xGF%_5v5,last_20_SH%,sum_rolling20_TOI_pp,sum_rolling20_xGF_pp,last20_pp_TOI_per_game,last20_xGF_per_min_pp,sum_rolling20_TOI_pk,sum_rolling20_xGA_pk,last20_pk_TOI_per_game,last20_xGA_per_min_pk,sum_rolling30_TOI_5v5,sum_rolling30_FF_5v5,sum_rolling30_FA_5v5,sum_rolling30_GF_5v5,sum_rolling30_GA_5v5,sum_rolling30_xGF_5v5,sum_rolling30_xGA_5v5,sum_rolling30_SF_5v5,last_30_FF%_5v5,last_30_GF%_5v5,last_30_xGF%_5v5,last_30_SH%,sum_rolling30_TOI_pp,sum_rolling30_xGF_pp,last30_pp_TOI_per_game,last30_xGF_per_min_pp,sum_rolling30_TOI_pk,sum_rolling30_xGA_pk,last30_pk_TOI_per_game,last30_xGA_per_min_pk
1511,"2021-04-29 - Flyers 3, Devils 5",PHI,Limited ReportFull Report,50.10,46,31,59.74,33,22,60.00,20,13,60.61,1,2,33.33,1.68,1.19,58.56,25,18,58.14,5,5,50.00,3,3,50.00,1,1,50.00,33.33,66.67,20,13,60.61,7,3,70.00,0,1,0.0,0.00,66.67,20,13,60.61,9,7,56.25,...,49.882145,6.048387,40.150000,5.62,4.015000,0.139975,51.966667,5.17,5.196667,0.099487,985.616667,701.0,604.0,29.0,43.0,34.97,32.90,521.0,53.716475,40.277778,51.524974,5.566219,89.766667,11.72,4.488333,0.130561,103.766667,9.97,5.188333,0.096081,1460.466667,1005.0,870.0,47.0,82.0,50.03,48.32,746.0,53.600000,36.434109,50.869344,6.300268,150.533333,16.17,5.017778,0.107418,142.083333,14.13,4.736111,0.099449
1509,"2021-04-29 - Sabres 2, Bruins 5",BUF,Limited ReportFull Report,50.10,40,48,45.45,32,43,42.67,27,30,47.37,2,3,40.00,1.98,1.57,55.75,22,16,57.89,7,8,46.67,7,6,53.85,2,2,50.00,28.57,66.67,15,8,65.22,9,5,64.29,0,1,0.0,0.00,80.00,16,30,34.78,10,17,37.04,...,46.831337,6.923077,53.616667,4.06,5.361667,0.075723,36.600000,5.59,3.660000,0.152732,977.350000,642.0,804.0,37.0,44.0,35.41,42.34,495.0,44.398340,45.679012,45.543408,7.474747,95.216667,7.09,4.760833,0.074462,91.316667,11.99,4.565833,0.131301,1492.183333,945.0,1171.0,53.0,76.0,52.51,64.06,698.0,44.659735,41.085271,45.045895,7.593123,122.783333,8.39,4.092778,0.068332,136.633333,17.86,4.554444,0.130715
1508,"2021-04-29 - Sabres 2, Bruins 5",BOS,Limited ReportFull Report,50.10,48,40,54.55,43,32,57.33,30,27,52.63,3,2,60.00,1.57,1.98,44.25,16,22,42.11,8,7,53.33,6,7,46.15,2,2,50.00,33.33,71.43,8,15,34.78,5,9,35.71,1,0,100.0,20.00,100.00,30,16,65.22,17,10,62.96,...,59.129173,8.041958,45.983333,3.32,4.598333,0.072200,68.583333,6.43,6.858333,0.093755,945.433333,729.0,600.0,42.0,35.0,35.77,30.12,548.0,54.853273,54.545455,54.287449,7.664234,102.383333,8.54,5.119167,0.083412,117.666667,9.77,5.883333,0.083031,1423.750000,1058.0,895.0,55.0,46.0,49.38,45.18,788.0,54.173067,54.455446,52.220812,6.979695,154.966667,12.35,5.165556,0.079695,171.583333,14.90,5.719444,0.086838
1519,"2021-04-29 - Panthers 4, Blackhawks 3",FLA,Limited ReportFull Report,46.95,51,29,63.75,40,23,63.49,27,19,58.70,2,2,50.00,2.20,1.30,62.91,24,11,68.57,7,5,58.33,5,5,50.00,1,2,33.33,20.00,60.00,17,6,73.91,7,5,58.33,1,0,100.0,14.29,100.00,25,16,60.98,14,9,60.87,...,57.146845,7.067138,66.150000,6.01,6.615000,0.090854,38.200000,4.13,3.820000,0.108115,950.083333,700.0,592.0,37.0,33.0,39.34,34.09,534.0,54.179567,52.857143,53.574833,6.928839,126.300000,12.59,6.315000,0.099683,88.483333,10.63,4.424167,0.120136,1406.050000,1019.0,907.0,59.0,54.0,58.57,51.79,780.0,52.907580,52.212389,53.071765,7.564103,193.533333,20.46,6.451111,0.105718,144.850000,15.11,4.828333,0.104315
1510,"2021-04-29 - Flyers 3, Devils 5",N.J,Limited ReportFull Report,5

In [532]:
feature_columns_all_seasons = ['Team_Key',
 'last_3_FF%_5v5',
 'last_3_GF%_5v5',
 'last_3_xGF%_5v5',
 'last_3_SH%',
 'last3_pp_TOI_per_game',
 'last3_xGF_per_min_pp',
 'last3_pk_TOI_per_game',
 'last3_xGA_per_min_pk',
 'B2B',
 'last_5_FF%_5v5',
 'last_5_GF%_5v5',
 'last_5_xGF%_5v5',
 'last_5_SH%',
 'last5_pp_TOI_per_game',
 'last5_xGF_per_min_pp',
 'last5_pk_TOI_per_game',
 'last5_xGA_per_min_pk',
 'last_10_FF%_5v5',
 'last_10_GF%_5v5',
 'last_10_xGF%_5v5',
 'last_10_SH%',
 'last10_pp_TOI_per_game',
 'last10_xGF_per_min_pp',
 'last10_pk_TOI_per_game',
 'last10_xGA_per_min_pk',
 'last_20_FF%_5v5',
 'last_20_GF%_5v5',
 'last_20_xGF%_5v5',
 'last_20_SH%',
 'last20_pp_TOI_per_game',
 'last20_xGF_per_min_pp',
 'last20_pk_TOI_per_game',
 'last20_xGA_per_min_pk',
 'last_30_FF%_5v5',
 'last_30_GF%_5v5',
 'last_30_xGF%_5v5',
 'last_30_SH%',
 'last30_pp_TOI_per_game',
 'last30_xGF_per_min_pp',
 'last30_pk_TOI_per_game',
 'last30_xGA_per_min_pk']

In [533]:
df_20172018_B = merge_starters_and_features(results, goalie_features_dfB, team_stats_all_seasons, feature_columns_all_seasons, goalie_feature_columns)
df_20182019_B = merge_starters_and_features(results1819, goalie_features_dfB, team_stats_all_seasons, feature_columns_all_seasons, goalie_feature_columns)
df_20192020_B = merge_starters_and_features(results1920, goalie_features_dfB, team_stats_all_seasons, feature_columns_all_seasons, goalie_feature_columns)
df_20202021_B = merge_starters_and_features(results2021, goalie_features_dfB, team_stats_all_seasons, feature_columns_all_seasons, goalie_feature_columns)

In [537]:
all_games_multirolling_noSVA = pd.concat([df_20172018_B, df_20182019_B, df_20192020_B, df_20202021_B])

In [538]:
#impute goalie stats where lack of games causing NaN
all_games_multirolling_noSVA['away_Last_20_FenwickSV%'] = np.where(all_games_multirolling_noSVA['away_Last_20_FenwickSV%'].isna(), ig_FenwickSV,all_games_multirolling_noSVA['away_Last_20_FenwickSV%'])
all_games_multirolling_noSVA['away_Last_20_GSAx/60'] = np.where(all_games_multirolling_noSVA['away_Last_20_GSAx/60'].isna(), ig_GSAx60, all_games_multirolling_noSVA['away_Last_20_GSAx/60'])
all_games_multirolling_noSVA['away_Last_20_HDCSV%'] = np.where(all_games_multirolling_noSVA['away_Last_20_HDCSV%'].isna(), ig_HDCSV, all_games_multirolling_noSVA['away_Last_20_HDCSV%'])
all_games_multirolling_noSVA['home_Last_20_FenwickSV%'] = np.where(all_games_multirolling_noSVA['home_Last_20_FenwickSV%'].isna(), ig_FenwickSV,all_games_multirolling_noSVA['home_Last_20_FenwickSV%'])
all_games_multirolling_noSVA['home_Last_20_GSAx/60'] = np.where(all_games_multirolling_noSVA['home_Last_20_GSAx/60'].isna(), ig_GSAx60, all_games_multirolling_noSVA['home_Last_20_GSAx/60'])
all_games_multirolling_noSVA['home_Last_20_HDCSV%'] = np.where(all_games_multirolling_noSVA['home_Last_20_HDCSV%'].isna(), ig_HDCSV, all_games_multirolling_noSVA['home_Last_20_HDCSV%'])

In [541]:
all_games_multirolling_noSVA.to_csv('data/all_games_multirolling_noSVA.csv')

In [540]:
pd.options.display.max_rows = 104
all_games_multirolling_noSVA.isna().sum()

game_id                         0
date                            0
venue                           0
home_team                       0
away_team                       0
start_time                      0
home_score                      0
away_score                      0
status                          0
Home_Team_Won                   0
Home_Team_Key                   0
Away_Team_Key                   0
home_goalie                    12
home_Last_20_FenwickSV%         0
home_Last_20_GSAx/60            0
home_Last_20_HDCSV%             0
away_goalie                    16
away_Last_20_FenwickSV%         0
away_Last_20_GSAx/60            0
away_Last_20_HDCSV%             0
home_Team_Key                   3
home_last_3_FF%_5v5             4
home_last_3_GF%_5v5             5
home_last_3_xGF%_5v5            4
home_last_3_SH%                 4
home_last3_pp_TOI_per_game      4
home_last3_xGF_per_min_pp       4
home_last3_pk_TOI_per_game      4
home_last3_xGA_per_min_pk       4
home_B2B      

### Get Data With Scoring and Venue Adjustments

In [40]:
#scraping team stats from NST
sequence = [x/10 for x in range(60, 120)]
time.sleep(random.choice(sequence))

primarysva1617 = get_and_format_nst_team_stats('20162017', 'sva', 'n')
primarysva1718 = get_and_format_nst_team_stats('20172018','sva', 'n')
primarysva1819 = get_and_format_nst_team_stats('20182019','sva', 'n')
primarysva1920 = get_and_format_nst_team_stats('20192020','sva', 'n')
primarysva2021 = get_and_format_nst_team_stats('20202021','sva', 'n')

time.sleep(random.choice(sequence))

pp1617 = get_and_format_nst_team_stats('20162017','pp', 'n')
pp1718 = get_and_format_nst_team_stats('20172018','pp', 'n')
pp1819 = get_and_format_nst_team_stats('20182019','pp', 'n')
pp1920 = get_and_format_nst_team_stats('20192020','pp', 'n')
pp2021 = get_and_format_nst_team_stats('20202021','pp', 'n')

time.sleep(random.choice(sequence))

pk1617 = get_and_format_nst_team_stats('20162017','pk', 'n')
pk1718 = get_and_format_nst_team_stats('20172018','pk', 'n')
pk1819 = get_and_format_nst_team_stats('20182019','pk', 'n')
pk1920 = get_and_format_nst_team_stats('20192020','pk', 'n')
pk2021 = get_and_format_nst_team_stats('20202021','pk', 'n')


In [41]:
#merge features for each season
featuressva1617 = merge_team_stats(primarysva1617, pp1617, pk1617)
featuressva1718 = merge_team_stats(primarysva1718, pp1718, pk1718)
featuressva1819 = merge_team_stats(primarysva1819, pp1819, pk1819)
featuressva1920 = merge_team_stats(primarysva1920, pp1920, pk1920)
featuressva2021 = merge_team_stats(primarysva2021, pp2021, pk2021)

In [42]:
#concat each season into one df
team_stats_all_seasons_sva = pd.concat([featuressva1617, featuressva1718, featuressva1819, featuressva1920, featuressva2021]).sort_values('Date')

In [43]:
#calculate different rolling game features
for games in [1,3,5,10,20,30,40,50]:
    team_stats_all_seasons_sva = calculate_team_features(team_stats_all_seasons_sva, games)

In [44]:
list(team_stats_all_seasons_sva.columns)

['Game',
 'Team',
 'Unnamed: 2',
 'TOI',
 'CF',
 'CA',
 'CF%',
 'FF',
 'FA',
 'FF%',
 'SF',
 'SA',
 'SF%',
 'GF',
 'GA',
 'GF%',
 'xGF',
 'xGA',
 'xGF%',
 'SCF',
 'SCA',
 'SCF%',
 'HDCF',
 'HDCA',
 'HDCF%',
 'HDSF',
 'HDSA',
 'HDSF%',
 'HDGF',
 'HDGA',
 'HDGF%',
 'HDSH%',
 'HDSV%',
 'MDCF',
 'MDCA',
 'MDCF%',
 'MDSF',
 'MDSA',
 'MDSF%',
 'MDGF',
 'MDGA',
 'MDGF%',
 'MDSH%',
 'MDSV%',
 'LDCF',
 'LDCA',
 'LDCF%',
 'LDSF',
 'LDSA',
 'LDSF%',
 'LDGF',
 'LDGA',
 'LDGF%',
 'LDSH%',
 'LDSV%',
 'SH%',
 'SV%',
 'PDO',
 'Attendance',
 'Date',
 'Game_Number',
 'Team_Key',
 'TOI_pk',
 'xGA_pk',
 'GA_pk',
 'TOI_pp',
 'xGF_pp',
 'GF_pp',
 'sum_rolling1_TOI_5v5',
 'sum_rolling1_FF_5v5',
 'sum_rolling1_FA_5v5',
 'sum_rolling1_GF_5v5',
 'sum_rolling1_GA_5v5',
 'sum_rolling1_xGF_5v5',
 'sum_rolling1_xGA_5v5',
 'sum_rolling1_SF_5v5',
 'last_1_FF%_5v5',
 'last_1_GF%_5v5',
 'last_1_xGF%_5v5',
 'last_1_SH%',
 'sum_rolling1_TOI_pp',
 'sum_rolling1_xGF_pp',
 'sum_rolling1_GF_pp',
 'last1_pp_TOI_per_game',
 'l

In [45]:
feature_columns_all_seasons = [
 'Game_Number',
 'Team_Key',
 'last_1_FF%_5v5',
 'last_1_GF%_5v5',
 'last_1_xGF%_5v5',
 'last_1_SH%',
 'last1_pp_TOI_per_game',
 'last1_xGF_per_min_pp',
 'last1_GF_per_min_pp',
 'last1_pk_TOI_per_game',
 'last1_xGA_per_min_pk',
 'last1_GA_per_min_pk',
 'last_3_FF%_5v5',
 'last_3_GF%_5v5',
 'last_3_xGF%_5v5',
 'last_3_SH%',
 'last3_pp_TOI_per_game',
 'last3_xGF_per_min_pp',
 'last3_GF_per_min_pp',
 'last3_pk_TOI_per_game',
 'last3_xGA_per_min_pk',
 'last3_GA_per_min_pk',
 'Last_Game_Date',
 'Days_Since_Last_Game',
 'B2B',
 'last_5_FF%_5v5',
 'last_5_GF%_5v5',
 'last_5_xGF%_5v5',
 'last_5_SH%',
 'last5_pp_TOI_per_game',
 'last5_xGF_per_min_pp',
 'last5_GF_per_min_pp',
 'last5_pk_TOI_per_game',
 'last5_xGA_per_min_pk',
 'last5_GA_per_min_pk',
 'last_10_FF%_5v5',
 'last_10_GF%_5v5',
 'last_10_xGF%_5v5',
 'last_10_SH%',
 'last10_pp_TOI_per_game',
 'last10_xGF_per_min_pp',
 'last10_GF_per_min_pp',
 'last10_pk_TOI_per_game',
 'last10_xGA_per_min_pk',
 'last10_GA_per_min_pk',
 'last_20_FF%_5v5',
 'last_20_GF%_5v5',
 'last_20_xGF%_5v5',
 'last_20_SH%',
 'last20_pp_TOI_per_game',
 'last20_xGF_per_min_pp',
 'last20_GF_per_min_pp',
 'last20_pk_TOI_per_game',
 'last20_xGA_per_min_pk',
 'last20_GA_per_min_pk',
 'last_30_FF%_5v5',
 'last_30_GF%_5v5',
 'last_30_xGF%_5v5',
 'last_30_SH%',
 'last30_pp_TOI_per_game',
 'last30_xGF_per_min_pp',
 'last30_GF_per_min_pp',
 'last30_pk_TOI_per_game',
 'last30_xGA_per_min_pk',
 'last30_GA_per_min_pk',
 'last_40_FF%_5v5',
 'last_40_GF%_5v5',
 'last_40_xGF%_5v5',
 'last_40_SH%',
 'last40_pp_TOI_per_game',
 'last40_xGF_per_min_pp',
 'last40_GF_per_min_pp',
 'last40_pk_TOI_per_game',
 'last40_xGA_per_min_pk',
 'last40_GA_per_min_pk',
'last40_pp_TOI_per_game',
 'last40_xGF_per_min_pp',
 'last40_GF_per_min_pp',
 'last40_pk_TOI_per_game',
 'last40_xGA_per_min_pk',
 'last40_GA_per_min_pk',
 'last_50_FF%_5v5',
 'last_50_GF%_5v5',
 'last_50_xGF%_5v5',
 'last_50_SH%',
 'last50_pp_TOI_per_game',
 'last50_xGF_per_min_pp',
 'last50_GF_per_min_pp',
 'last50_pk_TOI_per_game',
 'last50_xGA_per_min_pk',
 'last50_GA_per_min_pk']

In [46]:
#get official game results
results1718 = get_game_results('2017-10-04', '2018-04-08')
results1819 = get_game_results('2018-10-03', '2019-04-06')
results1920 = get_game_results('2019-10-02', '2020-03-12')
results2021 = get_game_results('2021-01-13', '2021-05-06')

Scraping the schedule between 2017-10-04 and 2018-04-08
Scraping the schedule between 2018-10-03 and 2019-04-06
Scraping the schedule between 2019-10-02 and 2020-03-12
Scraping the schedule between 2021-01-13 and 2021-05-06


In [80]:
display(results1718.shape)
display(results1819.shape)
display(results1920.shape)
display(results2021.shape)
display(results1718.shape[0] + results1819.shape[0] +results1920.shape[0] +results2021.shape[0])

(1271, 12)

(1272, 12)

(1083, 12)

(821, 12)

4447

In [47]:
df_20172018_C = merge_starters_and_features(results1718, goalie_features_dfC, team_stats_all_seasons_sva, elo, feature_columns_all_seasons, goalie_feature_columns)
df_20182019_C = merge_starters_and_features(results1819, goalie_features_dfC, team_stats_all_seasons_sva, elo, feature_columns_all_seasons, goalie_feature_columns)
df_20192020_C = merge_starters_and_features(results1920, goalie_features_dfC, team_stats_all_seasons_sva, elo, feature_columns_all_seasons, goalie_feature_columns)
df_20202021_C = merge_starters_and_features(results2021, goalie_features_dfC, team_stats_all_seasons_sva, elo, feature_columns_all_seasons, goalie_feature_columns)

In [48]:
all_games_multirolling_SVA_2 = pd.concat([df_20172018_C, df_20182019_C, df_20192020_C, df_20202021_C])

In [49]:
# some duplicates due to 2 goalies playsing > 28.5 minutes in a game, dropping first
all_games_multirolling_SVA_2 = all_games_multirolling_SVA_2[~all_games_multirolling_SVA_2.duplicated(subset='game_id')]

In [50]:
all_games_multirolling_SVA_2.shape

(4447, 206)

In [51]:
#all star game is only missing game from Elo ratings, will be dropped
all_games_multirolling_SVA_2[all_games_multirolling_SVA_2['home_Rating.A.Pre'].isna()]['home_team'].value_counts()

KHI                   1
AMERICAN ALL-STARS    1
Name: home_team, dtype: int64

In [ ]:
#impute goalie stats where lack of games causing NaN
all_games_multirolling_SVA_2['away_Goalie_FenwickSV%'] = np.where(all_games_multirolling_SVA_2['away_Goalie_FenwickSV%'].isna(), ig_FenwickSV,all_games_multirolling_SVA_2['away_Goalie_FenwickSV%'])
all_games_multirolling_SVA_2['away_Goalie_GSAx/60'] = np.where(all_games_multirolling_SVA_2['away_Goalie_GSAx/60'].isna(), ig_GSAx60, all_games_multirolling_SVA_2['away_Goalie_GSAx/60'])
all_games_multirolling_SVA_2['away_Goalie_HDCSV%'] = np.where(all_games_multirolling_SVA_2['away_Goalie_HDCSV%'].isna(), ig_HDCSV, all_games_multirolling_SVA_2['away_Goalie_HDCSV%'])
all_games_multirolling_SVA_2['home_Goalie_FenwickSV%'] = np.where(all_games_multirolling_SVA_2['home_Goalie_FenwickSV%'].isna(), ig_FenwickSV,all_games_multirolling_SVA_2['home_Goalie_FenwickSV%'])
all_games_multirolling_SVA_2['home_Goalie_GSAx/60'] = np.where(all_games_multirolling_SVA_2['home_Goalie_GSAx/60'].isna(), ig_GSAx60, all_games_multirolling_SVA_2['home_Goalie_GSAx/60'])
all_games_multirolling_SVA_2['home_Goalie_HDCSV%'] = np.where(all_games_multirolling_SVA_2['home_Goalie_HDCSV%'].isna(), ig_HDCSV, all_games_multirolling_SVA_2['home_Goalie_HDCSV%'])

In [64]:
all_games_multirolling_SVA_2.to_csv('data/all_games_multirolling_SVA_3.csv')

In [166]:
#not significant amount of games missing
pd.options.display.max_rows = 120
all_games_multirolling_SVA_2.isna().sum()[all_games_multirolling_SVA_2.isna().sum() >25]

Series([], dtype: int64)